# **PROCESO DE LIMPIEZA DE *SCRAPPING***

In [1]:
import pandas as pd # Manipulación df
import os # Directorios
import re # Manipulación de cadenas de texto (expresiones regulares)
import unicodedata# Función para eliminar acentos
from datetime import datetime, date
from currency_converter import CurrencyConverter # convertidor de monedas

# Cargar archivo

In [2]:
directorio_actual = os.getcwd() # Directorio actual de trabajo
# Ruta relativa al archivo CSV
csv_filename = 'lamudi-Abril-QueretaroInmuebles .xlsx'
csv_dir = os.path.join(directorio_actual)#, 'datos'
csv_path = os.path.join(csv_dir, csv_filename)

Leer archivo sin nombre en columnas:

In [ ]:
#column_names=['precio','denominacion','propiedad','metros_total','metros_construido','tiempo_de_publicacion','tipo','estacionamientos','recamaras','banos','medio_banos','seguridad_privada','ubicacion','url','descripcion']
df = pd.read_excel(csv_path)#, names=column_names
df.info()

Leer archivo con nombre de columnas:

### Limpieza de Nulos

In [4]:
df.fillna({'precio': 0}, inplace=True) 
df.fillna({'propiedad': 'Propiedad sin título'}, inplace=True) 
df.fillna({'metros_total': 0}, inplace=True) 
df.fillna({'metros_construido': 0}, inplace=True) 
df.fillna({'tiempo_de_publicacion': '1 ene 1000'}, inplace=True) 
df.fillna({'tipo': 'Propiedad sin tipo de casa'}, inplace=True) 
df.fillna({'estacionamientos': 0}, inplace=True) 
df.fillna({'recamaras': 0}, inplace=True) 
df.fillna({'banos': 0}, inplace=True) 
df.fillna({'medio_banos': 0}, inplace=True) 
df.fillna({'seguridad_privada': 'No'}, inplace=True) 
df.fillna({'ubicacion': 'Propiedad sin ubicación'}, inplace=True) 
df.fillna({'url': '-'}, inplace=True) 
df.fillna({'descripcion': "Propiedad sin descripcion"}, inplace=True) 

____
# 1. Manipulación de precios

## a) Precio en descripción

In [5]:
def extraer_precio(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones
    precios = re.findall(r'\b(\d+[\.,]?\d*[\.,]?\d*)\s*($|mil pesos|mxn|precio|pesos|monto|Pesos Mexicanos|USD)', descripcion)
    precio = None
    for match in precios:
        numero, moneda = match
        numero = numero.replace(',', '').replace("'", '').replace(".",'')
        try:
            precio = float(numero)
        except ValueError:
            continue
        if precio:
            break
    return precio

# Denominación de la descripción
def extraer_denominacion(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones 
    denominaciones = re.findall(r'\b(mil|mxn|precio|pesos|monto|Mexicanos|Estadounidenses)\b', descripcion)
    return denominaciones[0] if denominaciones else None

df['precio_desc'] = df['descripcion'].apply(extraer_precio)
#df['denominacion_desc'] = df['descripcion'].apply(extraer_denominacion)

### Comparaciones

#### Precio [0 / 0]

In [6]:
#posibles = df[( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 
# Configurar pandas para que no use notación científica en la impresión
#pd.set_option('display.float_format', '{:.2f}'.format)
# Imprimir las columnas especificadas del DataFrame filtrado
#print(posibles[['precio', 'denominacion', 'precio_desc', 'url' ]]) #'denominacion_desc'
# Restaurar la configuración original después de la impresión si es necesario
#pd.reset_option('display.float_format')

##### Proceso de reemplazo

In [7]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas
# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 o 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


Eliminar registros que no recuperaron su precio verdadero desde descripción

In [8]:
df = df[~( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 

#### Precio [0 / #]

In [9]:
#posibles = df[( (df['precio'] == 0 ) & (df['precio_desc'].notna()) )] 
# Configurar pandas para que no use notación científica en la impresión
#pd.set_option('display.float_format', '{:.2f}'.format)
# Imprimir las columnas especificadas del DataFrame filtrado
#print(posibles[['precio', 'denominacion', 'precio_desc', 'url' ]]) #'denominacion_desc'
# Restaurar la configuración original después de la impresión si es necesario
#pd.reset_option('display.float_format')

##### Proceso de reemplazo

In [10]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


#### Precio [# / #]

In [11]:
#posibles_cont = df[( (df['precio'].notna()) & (df['precio_desc'].notna()) )] 
# Filtro dentro del estándar
#sustituir = posibles_cont[(posibles_cont['precio_desc'] >= 250000) & (posibles_cont['precio_desc'] <= 500000000)]

# Configurar pandas para que no use notación científica en la impresión
#pd.set_option('display.float_format', '{:.2f}'.format)
# Imprimir las columnas especificadas del DataFrame filtrado
#print(sustituir[['precio', 'denominacion', 'precio_desc', 'url' ]]) #'denominacion_desc'
# Restaurar la configuración original después de la impresión si es necesario
#pd.reset_option('display.float_format')

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['precio']!= 0) & (df['precio_desc'].notna())) & (df['precio'] != df['precio_desc']) )] 
# Filtro dentro del estándar
totales = totales[(totales['precio_desc'] >= 250000) & (totales['precio_desc'] <= 500000000)]

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

## b) Denominación MXN

In [ ]:
df = pd.read_csv('abril_queretaro_Inmuebles_2024.csv')

SI EXISTE UNA DENOMINACIÓN QUE NO SEA MXN, SE CONVIERTA EL PRECIO

In [ ]:
import datetime

c = CurrencyConverter() # Instancia del convertidor de monedas
def convertir_a_mxn(precio, denominacion):
    if denominacion != 'MXN':
        return int(round(c.convert(precio, denominacion, 'MXN')))
    return precio

def convertir_a_usd(precio, denominacion):
    if denominacion != 'USD':
        return round(c.convert(precio, denominacion, 'USD'), 2)
    return precio

# Obtener la fecha actual
fecha_conversion = datetime.datetime.now().date()

# Aplicar la conversión a cada fila del DataFrame y agregar la fecha de conversión
df['precio_mxn'] = df.apply(lambda row: convertir_a_mxn(row['precio'], row['denominacion']), axis=1)
df['precio_usd'] = df.apply(lambda row: convertir_a_usd(row['precio'], row['denominacion']), axis=1)
df['denominacion_mxn'] = 'MXN'
df['denominacion_usd'] = 'USD'
df['fecha_conversion'] = fecha_conversion

print(df[['precio', 'denominacion', 'precio_mxn', 'denominacion_mxn', 'precio_usd', 'denominacion_usd', 'fecha_conversion']].head())

# Mover las columnas 'precio_mxn' y 'precio_usd' al inicio del DataFrame
#cols = ['precio_mxn', 'precio_usd'] + [col for col in df.columns if col not in ['precio_mxn', 'precio_usd']]
#df = df[cols]

## c) Precio fuera del estandar

IDENTIFICAR Y ELIMINAR PROPIEDADES QUE CUENTEN CON PRECIOS BAJOS y ALTOS AL ESTANDAR

In [ ]:
# Eliminar los registros donde el precio es menor al estandar (250,000) y con precio_desc vacío
df = df[(df['precio'] >= 250000)] #& (df['precio_desc'].notna())]
# Eliminar los registros donde el precio es mayor al estandar (500,000,000) y con precio_desc vacío
df = df[(df['precio'] <= 500000000)] #& (df['precio_desc'].notna())] 
# Ordenar valores de manera ascendente
df['precio'].sort_values()

## D) Segmento o categoría

In [ ]:
# Definir los rangos y las etiquetas
bins = [10, 500000, 750000, 1000000, 1250000, 1500000, 1750000, 2000000, 2250000, 2500000, 2750000, 3000000, 3250000, 3500000, 3750000, 100000000]
labels = ["E2", "E1", "D3", "D2", "D1", "C3", "C2", "C1", "B3", "B2", "B1", "A3", "A2", "A1", "S"]

# Utilizar pd.cut para clasificar los valores
df['categoria'] = pd.cut(df['precio'], bins=bins, labels=labels, right=False)
print(df['categoria'].unique())

# 2. Meses transcurridos

## a) Formato de la fecha

In [47]:
# Convertir mes en número
def month_to_int(month):
    months = ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sept', 'oct', 'nov', 'dic']
    return months.index(month.lower()) + 1

# Estandarizar el formato de fecha
def standardize_date(date_str):
    # Asegurarse de que date_str sea una cadena
    date_str = str(date_str)
    
    # Parsear formato "7 oct 2023"
    try:
        day, month, year = date_str.split(' ')
        month_int = month_to_int(month)
        return f"{int(day)} {month} {year}"
    except ValueError:
        pass
    
    # Parsear formato "07/03/2024"
    try:
        day, month, year = date_str.split('/')
        months = ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sept', 'oct', 'nov', 'dic']
        month_str = months[int(month) - 1]
        return f"{int(day)} {month_str} {year}"
    except ValueError:
        pass
    
    # Si no se puede parsear, devolver el string original 
    return date_str
df['tiempo_de_publicacion'] = df['tiempo_de_publicacion'].apply(standardize_date)


## b) Cálculo de meses

Sacar el número de meses transcurridos desde la fecha de su publicación a la actual al ejecutar este script.

In [ ]:
# Función para calcular los meses transcurridos
def months_since(date_str, today):
    parts = date_str.split()
    if len(parts) != 3:
        return None  # Return None for invalid date strings
    day, month, year = parts
    try:
        date = datetime(int(year), month_to_int(month), int(day))
        delta = today - date
        return delta.days // 30  # Approximate months
    except ValueError:
        return None  # Return None if date conversion fails

from datetime import datetime  # Import datetime class from datetime module

today = datetime.now()  # Obtener la fecha actual
current_date_str = today.strftime('%Y-%m-%d')

# Calcular los meses transcurridos, columna con la fecha actual en el nombre
new_column_name = f'meses_{current_date_str}'
df[new_column_name] = df['tiempo_de_publicacion'].apply(lambda x: months_since(x, today))
# Filtrar filas con valores None
df = df.dropna(subset=[new_column_name])
print(df[['tiempo_de_publicacion', new_column_name]])

# 3. Tipo de propiedad

## a) Lista de tipos deseados

In [ ]:
# Filtrar los registros según los tipos especificados
tipos_permitidos = 'Casa|Casa en Fraccionamiento|Casa en Condominio|Departamento|Penthouse|Dúplex|Loft|Estudio|Condominio Horizontal'
df = df[df['tipo'].str.contains(tipos_permitidos, na=False)]
df['tipo'].unique()

## b) Manejo de terrenos

Se buscan los registros que no sean las propiedades si no un solo terreno

In [ ]:
# Filtrar los registros según las condiciones dadas
totales =df[df['descripcion'].str.contains('lotes|terreno|Terreno|terrenos|Terrenos', case=False)]

# MANEJAR PROCESO DE ELIMINACIÓN 
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])#'descripcion',
    
    print("\n¿Te gustaría eliminar el registro?")
    # Solicitar la opción al usuario
    option = input("Elige una opción (si / no): ").strip().lower()
    
    if option == 'si':
        df.drop(idx, inplace=True)
        print(f"Registro eliminado.\n")
    else:
        print(f"Registro conservado.\n")
        print("\n---\n")
print("Proceso completado.")

# 4. Metros totales

## a) Metros en descripción

In [25]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|totales)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['totales_desc'] = df['descripcion'].apply(extraer_totales)

### Comparaciones

#### Metros [0 / 0]

In [26]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].isna() | df['totales_desc']==0) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


#### Metros [0 / #]

In [27]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


#### Metros [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_total']!= 0) & (df['totales_desc'].notna())) & (df['metros_total'] != df['totales_desc']) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(f"metros_total: {row['metros_total']}")
    print(f"totales_desc: {row['totales_desc']}")
    print(f"url: {row['url']}")
    
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        while True:
            try:
                new_value = float(input("Introduce el nuevo valor para 'metros_total': "))
                df.at[idx, 'metros_total'] = new_value
                break
            except ValueError:
                print("Por favor, introduce un número válido (entero o decimal).")
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


## b) Manejo por rangos de UMA

Rangos
* 118 UMAS = 40m2
* 118.1-200 = 50m2
* 200.1-350 = 71m2
* 350.1-750 = 102m2
* 750.1-1500 = 156m2

### DF cuando ambas columnas son 0 o vacíos

In [ ]:
# Rangos de UMAs y sus equivalencias en m²
umas_to_m2 = [
    (118, 40),
    (200, 50),
    (350, 71),
    (750, 102),
    (1500, 156)
]
valor_uma = 108.57

# Convertir MXN a UMAs
def precio_a_umas(precio, valor_uma):
    return precio / valor_uma

# Asignar m² basado en el precio en UMAs
def asignar_m2(precio_umas):
    for upper_bound, m2 in umas_to_m2:
        if precio_umas <= upper_bound:
            return m2
    return None  # En caso de que el precio en UMAs sea mayor al máximo definido

# Actualizar los registros con 'metros_total' igual a 0 y 'totales_desc' es NaN
def actualizar_metros_total(row):
    if row['metros_total'] == 0 and pd.isna(row['totales_desc']):
        precio_umas = precio_a_umas(row['precio'], valor_uma)
        row['metros_total'] = asignar_m2(precio_umas)
    return row

df = df.apply(actualizar_metros_total, axis=1)
print(df[['metros_total', 'totales_desc', 'url']])

# 5. M2 construidos

## a) M-construidos en descripción

In [30]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|construidos)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['construidos_desc'] = df['descripcion'].apply(extraer_totales)

### Comparaciones

#### Metros [0 / 0]

In [31]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].isna() | df['construidos_desc']==0) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


#### Metros [0 / #]

In [32]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


#### Metros [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_construido']!= 0) & (df['construidos_desc'].notna())) & (df['metros_construido'] != df['construidos_desc']) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        try:
            df.at[idx, 'metros_construido'] = float(new_value)
        except ValueError:
            print("Valor no válido. Debe ser un número. Pasando al siguiente registro...")
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


# 6. Precio / m2

## M2 Totales (terreno)

In [34]:
# Calcular precio por metro cuadrado del terreno
df['precio_m2_terreno'] = df['precio']/df['metros_total']

## M2 Construidos

In [35]:
# Calcular precio por metro cuadrado de la cosntrucción
df['precio_m2_construido'] = df['precio']/df['metros_construido']

# 7. Estacionamientos

## a) Estacionamientos descritos

In [36]:
# Función para extraer estacionamientos de la descripción
def extraer_coches(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(|cochera|garage|carros|carro|coche|coches|autos|automóviles|automóvil|auto|lugares de estacionamiento)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '').replace('.','')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['estacionamiento_desc'] = df['descripcion'].apply(extraer_coches)

### Estacionamientos [0 / 0]

In [38]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].isna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)    
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Estacionamientos [0 / #]

In [39]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Estacionamientos [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['estacionamientos']!=0) & (df['estacionamiento_desc'].notna())) & (df['estacionamientos'] != df['estacionamiento_desc']) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


## b) Mantener rango de estacionamientos < 5 (mínimo 1)

In [ ]:
df_filtrado = df[df['estacionamientos'] <= 5]
df_filtrado[['estacionamientos','estacionamiento_desc','url']]

In [ ]:
df = df[df['estacionamientos'] <= 5]

# 8. Recámaras

## a) Recámaras en descripción

In [59]:
# Función para extraer cantidad de recamaras en descripción
def extraer_recamaras(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(recamara|recamaras|habitaciones|dormitorio|dormitorios)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['recamaras_desc'] = df['descripcion'].apply(extraer_recamaras)

### Comparaciones

#### Recámaras [0 / 0]

In [53]:
# Filtrar los registros según las condiciones dadas
totales = df[ (df['recamaras'] == 0 ) & (df['recamaras_desc'].isna()) ] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


#### Recámaras [0 / #]

In [54]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']==0) & (df['recamaras_desc'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


#### Recámaras [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']!=0) & (df['recamaras_desc'].notna()) ) & (df['recamaras'] != df['recamaras_desc'])] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


## b) Mantener rango de recámaras <= 7 

In [ ]:
df['recamaras'].unique()

In [ ]:
dentro = df[(df['recamaras'] <= 7)] 
print(dentro[['recamaras','recamaras_desc','url']])

# 9. Baños

## a) Baños en descripción

Manejando tanto los completos como los medios

In [61]:
# Extraer la cantidad de baños y medios baños en la descripción
def extraer_baños(descripcion):
    descripcion = descripcion.lower()
    
    # Buscar patrones de baños completos
    completos = re.findall(r'\b(\d+[\.,]?\d*)\s*baños?\b', descripcion)
    total_completos = sum([float(match.replace(',', '').replace("'", '')) for match in completos])
    
    # Buscar patrones de medios baños (1/2 baño)
    medios = re.findall(r'\b(\d+[\.,]?\d*)\s*medios?\s*baños?\b', descripcion)
    total_medios = sum([float(match.replace(',', '').replace("'", '')) for match in medios])
    
    # Retornar como un diccionario o una tupla
    return {'completos': total_completos, 'medios': total_medios}

# Aplicar la función al DataFrame
df['baños_desc'] = df['descripcion'].apply(extraer_baños)

# Si deseas separar en dos columnas distintas:
df['baños_completos'] = df['baños_desc'].apply(lambda x: x['completos'] if x else 0)
df['medios_baños'] = df['baños_desc'].apply(lambda x: x['medios'] if x else 0)

### Baños

#### [0 / 0]

Poner como mínimo para el registro 1 en baños, pero si cuenta con medio baño en descripción que se le asigne ese valor.

In [62]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['banos'] == 0) & (df['baños_completos'].isna())]

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Proceso completado.


#### [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['banos']==0) & (df['baños_completos'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['banos']])
    print(row[['baños_completos']])
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['banos']!=0) & (df['baños_completos'].notna())) & (df['banos'] != df['baños_completos']))] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['banos']])
    print(row[['baños_completos']])    
    print(row[['url']])
    
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

### Medio baños

#### [0 / 0]

Poner como mínimo para el registro 1 en baños, pero si cuenta con valor en descripción que se le asigne ese valor.

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['medio_baños'] == 0) & (df['medios_baños'].isna())]

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

#### [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['medio_baños']==0) & (df['medios_baños'].notna()) )] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'medio_baños' por el de 'medios_baños'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_baños': ")
        df.at[idx, 'medio_baños'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_baños' por 'medios_baños'
        df.at[idx, 'medio_baños'] = df.at[idx, 'medios_baños']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['medio_baños']!=0) & (df['medios_baños'].notna())) & (df['medio_baños'] != df['medios_baños']))] 

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'medio_baños' por el de 'medios_baños'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_baños': ")
        df.at[idx, 'medio_baños'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_baños' por 'medios_baños'
        df.at[idx, 'medio_baños'] = df.at[idx, 'medios_baños']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

### B) Total de baños

In [ ]:
# Make the medio_baños and banos sum
# 'medio_baños', divididos entre 2 para representar medios
df['bano_total'] = df['banos'] + (df['medio_baños'] / 2)
df['bano_total'] = df['banos_totales'].round(1)
print(df[['banos', 'medio_baños', 'bano_total']].head())

# 10. Ubicación

## a) Desde descripción
#### Todavía en espera de estructura de código

## b) Separación

Se divide la ubicación por 
* Colonia
* Municipio
* Estado

In [ ]:
df[['Colonia', 'Municipio', 'Estado']] = df['ubicacion'].str.split(',', expand=True)

# Limpiar espacios en blanco alrededor de los valores resultantes
df['Colonia'] = df['Colonia'].str.strip()
df['Municipio'] = df['Municipio'].str.strip()
df['Estado'] = df['Estado'].str.strip()

# Sustituir NaN con valores específicos según la columna
df['Colonia'] = df['Colonia'].fillna('Sin_colonia')
df['Municipio'] = df['Municipio'].fillna('Sin_municipio')
df['Estado'] = df['Estado'].fillna('Sin_estado')

df[['ubicacion','Colonia','Municipio','Estado',]]

In [ ]:
df['Colonia'].unique()

## c) Estandarizar valores Colonia

In [43]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['Colonia'] = df['Colonia'].astype(str)
# Convertir a minúsculas
df['Colonia'] = df['Colonia'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')

# Aplicar la función de eliminación de acentos
df['Colonia'] = df['Colonia'].apply(eliminar_acentos)

# Reemplazar Ñ
df['Colonia'] = df['Colonia'].str.replace('ñ', 'n')
df['Colonia'] = df['Colonia'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

## d) Mapeo de CP por Excel

In [ ]:
# Cargar todas las hojas del archivo Excel en un diccionario de DataFrames
ruta_excel = 'D:\yoe11\Documents\TESEO\Modelo Factible\Scraplining\CPdescarga.xlsx'  
hojas = pd.read_excel(ruta_excel, sheet_name=None)

# Obtener combinaciones únicas de Estado, Municipio y Colonia
combinaciones_unicas = df[['Estado', 'Municipio', 'Colonia']].drop_duplicates()

# Crear un diccionario para almacenar los códigos postales encontrados
codigos_postales = {}

# Buscar el código postal para cada combinación única
for _, row in combinaciones_unicas.iterrows():
    estado = row['Estado'].strip()
    municipio = row['Municipio'].strip()
    colonia = row['Colonia'].strip()

    # Verificar si el estado existe en las hojas del Excel
    if estado in hojas:
        df_estado = hojas[estado]

        # Filtrar por municipio y colonia
        df_filtrado = df_estado[
            (df_estado['D_mnpio'].str.strip().str.lower() == municipio.lower()) &
            (df_estado['d_asenta'].str.strip().str.lower() == colonia.lower())  # Suponiendo que la columna de colonia se llama 'd_asenta'
        ]

        # Si hay coincidencias, almacenar el primer código postal en el diccionario
        if not df_filtrado.empty:
            codigos_postales[(estado, municipio, colonia)] = df_filtrado['d_codigo'].iloc[0]
        else:
            codigos_postales[(estado, municipio, colonia)] = None
    else:
        codigos_postales[(estado, municipio, colonia)] = None

# Crear una columna 'Codigo_Postal' en el DataFrame original usando el diccionario de códigos postales
df['Codigo_Postal'] = df.apply(lambda x: codigos_postales.get((x['Estado'].strip(), x['Municipio'].strip(), x['Colonia'].strip())), axis=1)


# Obtener las combinaciones únicas de las columnas 'Colonia', 'Municipio', 'Estado', 'Codigo_Postal'
combinaciones_unicas = df[['Colonia', 'Municipio', 'Estado', 'Codigo_Postal']].drop_duplicates()
for idx, row in combinaciones_unicas.iterrows():
    print(f"Combinación {idx + 1}:")
    print(f"Colonia: {row['Colonia']}, Municipio: {row['Municipio']}, Estado: {row['Estado']}, Codigo_Postal: {row['Codigo_Postal']}")
    print("---")


## e) Fraccionamientos

In [ ]:
# Traer de la base original de los cp, identificar la colonia si es fraccionamiento, 
# si lo verifica que se le asigne un valor para identificar cuál es cual.

# Cargar todas las hojas del archivo Excel en un diccionario de DataFrames
ruta_excel = 'D:\yoe11\Documents\TESEO\Modelo Factible\Scraplining\CPdescarga.xlsx'  
hojas = pd.read_excel(ruta_excel, sheet_name=None)

# Obtener combinaciones únicas de Estado, Municipio y Colonia
combinaciones_unicas = df[['Estado', 'Municipio', 'Colonia']].drop_duplicates()

# Crear un diccionario para almacenar los códigos postales y tipos de asentamiento encontrados
codigos_postales = {}
tipos_asentamiento = {}

# Buscar el código postal y tipo de asentamiento para cada combinación única
for _, row in combinaciones_unicas.iterrows():
    estado = row['Estado'].strip()
    municipio = row['Municipio'].strip()
    colonia = row['Colonia'].strip()

    # Verificar si el estado existe en las hojas del Excel
    if estado in hojas:
        df_estado = hojas[estado]

        # Filtrar por municipio y colonia
        df_filtrado = df_estado[
            (df_estado['D_mnpio'].str.strip().str.lower() == municipio.lower()) &
            (df_estado['d_asenta'].str.strip().str.lower() == colonia.lower())
        ]

        # Si hay coincidencias, almacenar el primer código postal y tipo de asentamiento en los diccionarios
        if not df_filtrado.empty:
            codigos_postales[(estado, municipio, colonia)] = df_filtrado['d_codigo'].iloc[0]
            tipos_asentamiento[(estado, municipio, colonia)] = df_filtrado['d_tipo_asenta'].iloc[0]
        else:
            codigos_postales[(estado, municipio, colonia)] = None
            tipos_asentamiento[(estado, municipio, colonia)] = None
    else:
        codigos_postales[(estado, municipio, colonia)] = None
        tipos_asentamiento[(estado, municipio, colonia)] = None

# Crear columnas 'Codigo_Postal' y 'Tipo_Asentamiento' en el DataFrame original usando los diccionarios
df['Codigo_Postal'] = df.apply(lambda x: codigos_postales.get((x['Estado'].strip(), x['Municipio'].strip(), x['Colonia'].strip())), axis=1)
df['Tipo_Asentamiento'] = df.apply(lambda x: tipos_asentamiento.get((x['Estado'].strip(), x['Municipio'].strip(), x['Colonia'].strip())), axis=1)

# Obtener las combinaciones únicas de las columnas 'Colonia', 'Municipio', 'Estado', 'Codigo_Postal', 'Tipo_Asentamiento'
combinaciones_unicas = df[['Colonia', 'Municipio', 'Estado', 'Codigo_Postal', 'Tipo_Asentamiento']].drop_duplicates()
for idx, row in combinaciones_unicas.iterrows():
    print(f"Combinación {idx + 1}:")
    print(f"Colonia: {row['Colonia']}, Municipio: {row['Municipio']}, Estado: {row['Estado']}, Codigo_Postal: {row['Codigo_Postal']}, Tipo_Asentamiento: {row['Tipo_Asentamiento']}")
    print("---")

___
# Eliminar variables extra

In [ ]:
df = df.drop(['precio_desc','totales_desc','construidos_desc','estacionamiento_desc','recamaras_desc','baños_desc','baños_completos','medios_baños'], axis=1) # 'denominacion_desc'

# EXPORTAR

In [51]:
# Lista de nombres de columnas en el orden deseado
column_order = ['precio','denominacion','propiedad','metros_total','metros_construido','tiempo_de_publicacion','meses_2024-09-06', 'tipo', 'estacionamientos', 'recamaras', 'banos','medio_banos','seguridad_privada', 'Estado','Municipio','Colonia',  'Codigo_Postal','url','descripcion']
# Reordenar el DataFrame según el orden de columnas deseado
df = df[column_order]

df.to_csv('abril_queretaro_Inmuebles_2024.csv')